In [1]:
import pyrcn

In [2]:
import os

In [3]:
os.listdir('/Volumes/PassportDrive')

['.DS_Store',
 '.fseventsd',
 '.Spotlight-V100',
 '.TemporaryItems',
 '.Trashes',
 'IDRad',
 'Install Western Digital Software for Mac.dmg',
 'Install Western Digital Software for Windows.exe',
 'Vimeo']

In [3]:
# DATA_DIR = '/Volumes/Untitled/IDRad/idrad'
DATA_DIR = '/Users/davidzhu/Desktop/IDRad/idrad'
DEFAULT_FILE = 'train/target5_001.hdf5'

# Preprocessing

In [4]:
import h5py
import numpy as np 

In [5]:
def range_doppler(data, chirps=256,
                  samples=256,
                  fft_rangesamples=2 ** 10,
                  fft_dopplersamples=2 ** 8,
                  fs=2.0e6,
                  kf=1171875.0e7,
                  min_range=0.5,
                  max_range=10):
    """
    Computes a range-doppler map for a given number of chirps and samples per chirp.
    :param data: FMCW radar data frame consisting of <chirps>x<samples>
    :param chirps: Number of chirps (Np)
    :param samples: Number of samples (N)
    :param fft_rangesamples: Number of samples for the range fft.
    :param fft_dopplersamples: Number of samples for the doppler fft.
    :param fs: Constant depending on the radar recording parameters.
    :param kf: Constant depending on the radar recording parameters.
    :param min_range: Minimum value to take into account for the range axis in the range-doppler map.
    :param max_range: Maximum value to take into account for the range axis in the range-doppler map.
    :return: Returns a 2D dimensional range-doppler map representing the reflected power over all range-doppler bins.
    """

    data = data.reshape(chirps, samples).T
    # Ignore chirp sequence number
    data = data[1:]
    Ny, Nx = data.shape  # rows (N), columns (Np)

    window = np.hanning(Ny)
    scaled = np.sum(window)
    window2d = np.tile(window, (Nx, 1)).T
    data = data * window2d

    # Calculate Range FFT
    x = np.zeros((fft_rangesamples, Nx))
    start_index = int((fft_rangesamples - Ny) / 2)
    x[start_index:start_index + Ny, :] = data
    X = np.fft.fft(x, fft_rangesamples, 0) / scaled * (2.0 / 2048)
    # Extract positive range bins
    X = X[0:fft_rangesamples // 2, :]
    # Extract range
    _freq = np.arange(fft_rangesamples // 2) / float(fft_rangesamples) * fs
    _range = _freq * 3e8 / (2 * kf)
    min_index = np.argmin(np.abs(_range - min_range))
    max_index = np.argmin(np.abs(_range - max_range))

    X = X[min_index: max_index, :]

    # Calculate Doppler FFT
    Ny, Nx = X.shape
    window = np.hanning(Nx)
    scaled = np.sum(window)
    window2d = np.tile(window, (Ny, 1))
    X = X * window2d

    rd = np.zeros((Ny, fft_dopplersamples), dtype='complex_')
    start_index = int((fft_dopplersamples - Nx) / 2)
    rd[:, start_index:start_index + Nx] = X

    range_doppler = np.fft.fft(rd, fft_dopplersamples, 1) / scaled
    range_doppler = np.fft.fftshift(range_doppler, axes=1)

    return np.abs(range_doppler)

def preprocess_file(fname): 
    with h5py.File(f'{DATA_DIR}/{fname}', 'r+') as file:
        nframes = file['radar'].shape[0]

        # Create datasets
        if not 'microdoppler' in file:
            file.create_dataset("microdoppler", (nframes, 256), dtype='float32', chunks=(1, 256))
        if not 'microdoppler_thresholded' in file:
            file.create_dataset("microdoppler_thresholded", (nframes, 256), dtype='float32', chunks=(1, 256))
        if not 'range_doppler' in file:
            file.create_dataset("range_doppler", (nframes, 380, 256), dtype='float32', chunks=True)

        
        x = file['range_doppler'][:10,:,:]
        
        #has not been preprocessed
        if np.all(x==0): 
            print('preprocessing')
        
            # Run over each radar frame
            for i in range(nframes): # only take first 1000 
                rd = range_doppler(file['radar'][i]) 
                rd = 20 * np.log10(rd)

                file['range_doppler'][i] = rd
                file['microdoppler'][i] = rd.sum(axis=0)

                rd -= np.amax(rd)
                rd[rd < -45] = -45
                file['microdoppler_thresholded'][i] = rd.sum(axis=0)

                if not i%100: 
                    print("Finished frame %d of %d." % (i + 1, nframes))
                    
def get_range_doppler(fname): 
    '''returns the range doppler'''
    preprocess_file(fname) 
    range_doppler = 0 
    
    with h5py.File(f'{DATA_DIR}/{fname}', 'r+') as file:
        # d['microdoppler'] = file['microdoppler'][:,:]
        # d['microdoppler_thresholded'] = file['microdoppler_thresholded'][:,:]
        range_doppler = file['range_doppler'][:,:,:].sum(axis=1)
    
    return range_doppler

In [68]:
dataset_files = os.listdir(f'{DATA_DIR}/train')
dataset = [] 
labels = []
labels_repeat = [] 

for fname in dataset_files: 
    if fname[:2] == '._':
        print(fname)
        continue 
    labels.append(int(fname[6]))
    dataset.append(get_range_doppler(f'train/{fname}'))
    labels_repeat.append(np.repeat(int(fname[6]), dataset[-1].shape[0] ))

# dataset = np.array(dataset) 
labels = np.array(labels)
labels_repeat = np.array(labels_repeat, dtype = object)
dataset = np.array(dataset, dtype = object)

In [69]:
dataset = np.array(dataset, dtype = object)

In [70]:
print(dataset.shape)
print(labels.shape)
print(labels_repeat.shape)

(130,)
(130,)
(130,)


In [9]:
labels_hot = np.zeros((labels.size, dataset.shape[1], labels.max()))
labels_hot[np.arange(labels.size),:,labels-1] = 1

In [10]:
from os.path import isfile

# Layer Making

In [11]:
from pyrcn.base.blocks import InputToNode, BatchIntrinsicPlasticity, NodeToNode, HebbianNodeToNode

In [12]:
from pyrcn.echo_state_network import ESNRegressor, ESNClassifier

In [13]:
from pyrcn.base.blocks import InputToNode, PredefinedWeightsInputToNode, NodeToNode
from pyrcn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error
from pyrcn.model_selection import SequentialSearchCV

from sklearn.base import clone
from sklearn.metrics import make_scorer, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, ParameterGrid
from sklearn.cluster import MiniBatchKMeans
from sklearn.utils.fixes import loguniform
from scipy.stats import uniform

In [20]:
default_rd = get_range_doppler(DEFAULT_FILE)

In [15]:
layer1_size = 125
layer2_size = 500

In [59]:
# Bilayered; i2n -> n2o, second layer has fewer (125) nodes

i2n = InputToNode(hidden_layer_size=layer1_size, input_activation="tanh", input_scaling=1.0, bias_scaling=0.1)

# layer1 = i2n.fit_transform(dataset)

In [60]:
layer1.shape

NameError: name 'layer1' is not defined

In [61]:
initial_params = {'hidden_layer_size': layer2_size, #1000 in 5C
                  'input_activation': 'identity', #
                  # 'k_in': layer1_size, # number of inputs 
                  'bias_scaling': 0.0, # usually 0 
                  'spectral_radius' : 0.0, 
                  'reservoir_activation': 'tanh', # 2
                  'leakage': 0.05, #equation 6, 17 frames should be 1 step 
                  'bidirectional': True, #bidirectional 
                  'k_rec': 10,
                  'alpha': 1e-5,
                  'random_state': 1,
                  'requires_sequence': True}

In [62]:
n2o = ESNClassifier(**initial_params)

In [73]:
n2o.fit(dataset, labels_repeat)

ESNClassifier(input_to_node=InputToNode(bias_scaling=0.0,
                                        input_activation='identity',
                                        random_state=1),
              node_to_node=NodeToNode(bidirectional=True, k_rec=10,
                                      leakage=0.05, random_state=1,
                                      sparsity=0.02, spectral_radius=0.0),
              regressor=IncrementalRegression(), requires_sequence=True)

In [74]:
predictions = n2o.predict(dataset)

In [86]:
correct = []
for i, prediction in enumerate(predictions):  
    
    correct.append((prediction == labels[i]).mean())

correct = np.array(correct)

In [87]:
correct.mean()

0.32139989854259865

In [88]:
# model from multipitch tracking examples 

initially_fixed_params = {'hidden_layer_size': 500,
                          'input_activation': 'identity',
                          'k_in': 10,
                          'bias_scaling': 0.0,
                          'reservoir_activation': 'tanh',
                          'leakage': 1.0,
                          'bidirectional': False,
                          'k_rec': 10,
                          'alpha': 1e-5,
                          'random_state': 42,
                          'requires_sequence': True}

step1_esn_params = {'leakage': np.linspace(0.1, 1.0, 10)}
kwargs_1 = {'random_state': 42, 'verbose': 2, 'n_jobs': 70, 'pre_dispatch': 70, 'n_iter': 14,
           'scoring': make_scorer(accuracy_score)}
step2_esn_params = {'input_scaling': np.linspace(0.1, 1.0, 10),
                    'spectral_radius': np.linspace(0.0, 1.5, 16)}

step3_esn_params = {'bias_scaling': np.linspace(0.0, 2.0, 21)}

kwargs_2_3 = {'verbose': 2, 'pre_dispatch': 70, 'n_jobs': 70, 
              'scoring': make_scorer(accuracy_score)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', GridSearchCV, step1_esn_params, kwargs_1),
            ('step2', GridSearchCV, step2_esn_params, kwargs_2_3),
            ('step3', GridSearchCV, step3_esn_params, kwargs_2_3)]

base_esn = ESNClassifier(**initially_fixed_params)



In [95]:
base_esn.fit(dataset, labels_repeat)

ESNClassifier(input_to_node=InputToNode(bias_scaling=0.0,
                                        input_activation='identity', k_in=10,
                                        sparsity=0.0390625),
              node_to_node=NodeToNode(k_rec=10, sparsity=0.02),
              regressor=IncrementalRegression(), requires_sequence=True)

In [96]:
predictions = base_esn.predict(dataset)

In [97]:
correct = []
for i, prediction in enumerate(predictions):  
    
    correct.append((prediction == labels[i]).mean())

correct = np.array(correct)

In [98]:
correct.mean()

0.295484848134971